# BSRC: Summer 2019 Research Project

## Connecting to a Redis Database and Processing Messages

This notebook will be used for sending and receiving messages through a Redis channel. The MeerKAT Breakthrough Listen Backend operates by sending messages through a Redis server over channels. This code will have to subscribe to multiple alert channels and act on the information passed along.

### Connecting to Redis Channel

The goal of this notebook will be to test connecting to databases and redis channels and act on the information sent over. I have a sample schedule block that I will publish to a redis channel and process by listening to that same channel. Most of the code that I've written is stored under the poorly named query_db.py and connector.py files. Here connections to the MySQL databases and redis channels, as well as the messages that pass over these channels, are handled. I'll start by importing these modules.

```
>>> redis-server
```

Then, in another terminal window, run the following command:

```
>>> python target_selector_start.py
```

In [13]:
import json
import redis
import numpy as np
import yaml
import time
import re

### Using Example Message

Most of the information that will be received from the Redis server in specific channels. The channels will send over strings of information pertinent to the observation. In this case, the schedule_block (or targets) channel will contain a string formatted like a dictionary. For the purposes of testing, I can load in an example schedule block message that will be similar to one that will be sent over a redis channel later. I'll start by connecting to the Redis server and loading in the sample message.

In [2]:
with open('../test/channels.txt', 'r') as f:
    channel = f.read()
    
with open('../test/messages.txt', 'r') as f:
    messages = f.read()

In [3]:
chnls = channel.split('\n')
msgs = messages.split('\n')

In [4]:
len(msgs), len(chnls)

(54, 54)

In [5]:
def publish_key(chan, key, val):
    r.set(key, val)
    r.publish(chan, key)
    return True

In [6]:
msgs

['configure:array_1',
 'conf_complete:array_1',
 'DESTIP=234.1.21.1+0',
 'DESTIP=234.1.21.2+0',
 'DESTIP=234.1.21.3+0',
 'DESTIP=234.1.21.4+0',
 'DESTIP=234.1.21.5+0',
 'DESTIP=234.1.21.6+0',
 'DESTIP=234.1.21.7+0',
 'DESTIP=234.1.21.8+0',
 'BINDPORT=1234',
 'FENSTRM=8',
 'FENCHAN=4096',
 'HNCHAN=512',
 'HNTIME=256',
 'capture-init:array_1',
 'array_1:target:radec, 0:00:00.00, -90:00:00.0',
 'array_1:subarray_1_pool_resources',
 'capture-start:array_1',
 'NETSTAT=RECORD',
 'm001_data_suspect:True',
 'array_1:data_suspect:True',
 'm005_data_suspect:True',
 'array_1:data_suspect:True',
 'm012_data_suspect:True',
 'array_1:data_suspect:True',
 'm015_data_suspect:True',
 'array_1:data_suspect:True',
 'm001_data_suspect:False',
 'array_1:data_suspect:True',
 'm001_data_suspect:True',
 'array_1:data_suspect:True',
 'm005_data_suspect:False',
 'array_1:data_suspect:True',
 'm012_data_suspect:False',
 'array_1:data_suspect:True',
 'm015_data_suspect:False',
 'array_1:data_suspect:True',
 'm001

In [7]:
r = redis.StrictRedis()

In [8]:
pool_resources = 'bluse_1,cbf_1,m001,m005,m012,m015,sdp_1'

In [9]:
publish_key('sensor_alerts', 'array_1:subarray_1_pool_resources', pool_resources)

True

In [19]:
for i in range(len(msgs)-3):
    if msgs[i].startswith('m0'):
        continue
    r.publish(chnls[i], msgs[i])
    time.sleep(0.05)

In [10]:
','.join(re.findall('m\d\d\d', pool_resources))

'm001,m005,m012,m015'

In [11]:
re.findall('m\d{3}', pool_resources)

['m001', 'm005', 'm012', 'm015']

In [13]:
re.findall('[a-z A-Z]+_\d', pool_resources)

['bluse_1', 'cbf_1', 'sdp_1']